In [1]:
# import packages
import os
import cv2
import numpy as np;
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import torch 
import torch.nn as nn
import torchvision.transforms.functional as TF
import segmentation_models_pytorch as smp
import albumentations as album
import math

from sklearn import model_selection
from scipy import ndimage
from typing import Tuple, List
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from torch.utils.data import DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import BCELoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_
import joblib
import time
import wandb
import random
import itertools

class CaImagesDataset(Dataset):
    """CA Images dataset."""
    # load the dataset
    def __init__(self, x, y):
        self.x = torch.tensor(x,dtype=torch.float32)
        self.y = torch.tensor(y,dtype=torch.float32)

    # number of samples in the dataset
    def __len__(self):
        return len(self.x)
    
    # get a sample from the dataset
    def __getitem__(self, idx):
        return [self.x[idx], self.y[idx]]
    
class NeuralNetworkClassifier(nn.Module):
    """Neural network with LSTM layer and fully connected layer"""
    def __init__(self):
        super(NeuralNetworkClassifier,self).__init__()
        self.lstm = nn.LSTM(input_size=2, 
                            hidden_size=2,
                            bidirectional=False,
                            num_layers=1,
                            batch_first=True
                            )

    def forward(self,x):
        output,_status = self.lstm(x)
        return output


In [2]:

data_dir = "/Users/huayinluo/Desktop/code/catracking-1/data"
video_dir = os.path.join(data_dir, "imgs")
position_dir = os.path.join(data_dir, "positions")
model_dir = "/Users/huayinluo/Desktop/code/catracking-1/models/lstm"
img_dir = "/Users/huayinluo/Desktop/code/catracking-1/images"
results_dir = "/Users/huayinluo/Desktop/code/catracking-1/results"

In [16]:
video = "11408"
AVA_positions = np.load(os.path.join(position_dir, f"AVA_{video}.mat.npy"))
AVB_positions = np.load(os.path.join(position_dir, f"AVB_{video}.mat.npy"))
AVA_positions.shape, AVB_positions.shape
all_positions = np.stack((AVA_positions, AVB_positions))
all_positions.shape

(2, 2571, 2)

In [18]:
all_positions[0]

array([[117.        ,  92.5       ],
       [118.08333333,  87.66666667],
       [117.5       ,  82.5       ],
       ...,
       [132.05882353, 302.41176471],
       [130.23076923, 302.84615385],
       [130.29166667, 302.70833333]])

In [19]:
all_positions[1]

array([[109.5       , 108.        ],
       [111.        , 105.5       ],
       [110.5       , 100.875     ],
       ...,
       [135.5       , 314.5       ],
       [135.5       , 315.5       ],
       [136.66666667, 316.33333333]])

In [59]:

# Save all video arrays and positions in dictionary
videos = ['11408', '11409', "11410", '11411']
positions_dct={}
for video in videos:
  AVA_positions = np.load(os.path.join(position_dir, f"AVA_{video}.mat.npy"))
  AVB_positions = np.load(os.path.join(position_dir, f"AVB_{video}.mat.npy"))
  all_neurons_positions = np.stack((AVA_positions, AVB_positions))
  positions_dct[video] = all_neurons_positions
  print(f"Loading {video}...")
print(f"Finished loading images and positions: {len(imgs_dct)} videos, {len(positions_dct)} positions")


Loading 11408...
Loading 11409...
Loading 11410...
Loading 11411...
Finished loading images and positions: 7 videos, 4 positions


In [65]:

# Test/train split (# Add 80% of each video to training set, 20% to testing set)
df_train_lst = []
df_test_lst = []
for video in videos:
  all_positions = positions_dct[video]
  height, width = cv2.imread(f"/Users/huayinluo/Desktop/code/catracking-1/images/original/{video}/0.png").shape[:2]
  norm_positions = np.multiply(all_positions, [1/width, 1/height])
  split = math.floor(norm_positions.shape[1]*0.8)
  df_train_lst.append(norm_positions[:, :split, :])
  df_test_lst.append(norm_positions[:, split:, :])
print("Test/Train split complete")

Test/Train split complete


In [71]:
len(df_train_lst[0][0])

2056

In [72]:
ava, avb = df_train_lst[0]

In [74]:
ava.shape, avb.shape

((2056, 2), (2056, 2))

In [77]:
ava[:2], avb[:2]

(array([[0.31034483, 0.1911157 ],
        [0.31321839, 0.18112948]]),
 array([[0.29045093, 0.2231405 ],
        [0.29442971, 0.21797521]]))

In [83]:
np.zeros((2, 2))

array([[0., 0.],
       [0., 0.]])

In [89]:
lst = np.zeros(10)
lst[:2] = np.ones(2)
lst

array([1., 1., 0., 0., 0., 0., 0., 0., 0., 0.])

In [110]:
n=5
lst = np.concatenate((np.zeros(2), np.ones(n-2)))
print(lst)
random.shuffle(lst)

print(lst)

[0. 0. 1. 1. 1.]
[1. 1. 0. 1. 0.]


In [100]:
all_lst = []
for i in range(2):
    lst = np.zeros((2, 2))
    for j in range
    lst[i] = ava[i]

SyntaxError: expected ':' (2846566804.py, line 4)

In [86]:
lst

array([[0.31034483, 0.1911157 ],
       [0.31321839, 0.18112948]])